# 3.3 Image Captioning with Transformers

你已经实现了一个普通的 RNN 用于图像描述生成任务。在本笔记本中，你将实现 Transformer 解码器的关键部分来完成相同的任务。

**注意：** 与 RNN 笔记本不同，本笔记本将主要使用 PyTorch 而非 NumPy。

In [ ]:
# Setup cell.
import time, os, json
import numpy as np
import matplotlib.pyplot as plt

from mml.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from mml.transformer_layers import *
from mml.captioning_solver_transformer import CaptioningSolverTransformer
from mml.classifiers.transformer import CaptioningTransformer
from mml.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from mml.image_utils import image_from_url

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # Set default size of plots.
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# COCO 数据集

与之前的笔记本一样，我们将使用 COCO 数据集进行图像描述生成任务。

In [ ]:
# Load COCO data from disk into a dictionary.
data = load_coco_data(pca_features=True)

# Print out all the keys and values from the data dictionary.
for k, v in data.items():
    if type(v) == np.ndarray:
        print(k, type(v), v.shape, v.dtype)
    else:
        print(k, type(v), len(v))

# Transformer

正如你所见，RNN 非常强大，但通常训练速度较慢。此外，RNN 在编码长程依赖方面存在困难（尽管 LSTM 是缓解该问题的一种方法）。2017 年，Vaswani 等人在论文 ["Attention Is All You Need"](https://arxiv.org/abs/1706.03762) 中引入了 Transformer，旨在 a) 引入并行化，b) 让模型学习长程依赖。这篇论文不仅催生了 NLP 领域著名的模型如 BERT 和 GPT，还引发了各个领域的广泛兴趣，包括计算机视觉。尽管在这里我们将模型引入图像描述生成任务的上下文中，但注意力机制的思想本身具有更广泛的适用性。

# Transformer: 多头注意力机制

### 点积注意力 (Dot-Product Attention)

回想一下，注意力机制可以被视为对查询向量 $q\in\mathbb{R}^d$、一组值向量 $\{v_1,\dots,v_n\}, v_i\in\mathbb{R}^d$ 和一组键向量 $\{k_1,\dots,k_n\}, k_i \in \mathbb{R}^d$ 的操作，其定义如下：

\begin{align}
c = \sum_{i=1}^{n} v_i \alpha_i &\alpha_i = \frac{\exp(k_i^\top q)}{\sum_{j=1}^{n} \exp(k_j^\top q)} \\
\end{align}

其中 $\alpha_i$ 通常被称为“注意力权重”，输出 $c\in\mathbb{R}^d$ 是值向量的加权平均。

### 自注意力 (Self-Attention)
在 Transformer 中，我们执行自注意力，这意味着值向量、键向量和查询向量均源自输入 $X \in \mathbb{R}^{\ell \times d}$，其中 $\ell$ 为序列长度。具体而言，我们学习参数矩阵 $V, K, Q \in \mathbb{R}^{d\times d}$ 来映射输入 $X$，如下所示：

\begin{align}
v_i = Vx_i\ \ i \in \{1,\dots,\ell\}\\
k_i = Kx_i\ \ i \in \{1,\dots,\ell\}\\
q_i = Qx_i\ \ i \in \{1,\dots,\ell\}
\end{align}

### 多头缩放点积注意力 (Multi-Headed Scaled Dot-Product Attention)

在多头注意力的情况下，我们为每个头学习一个参数矩阵，使模型能够更灵活地关注输入的不同部分。设 $h$ 为头的数量，$Y_i$ 为第 $i$ 个头的注意力输出。因此，我们分别学习单独的矩阵 $Q_i$、$K_i$ 和 $V_i$。为了使整体计算与单头注意力的情况相同，我们选择 $Q_i \in \mathbb{R}^{d\times d/h}$，$K_i \in \mathbb{R}^{d\times d/h}$ 和 $V_i \in \mathbb{R}^{d\times d/h}$。

在上述简单点积注意力基础上添加缩放因子 $\frac{1}{\sqrt{d/h}}$，公式如下：

$$
Y_i = \text{softmax}\bigg(\frac{(XQ_i)(XK_i)^\top}{\sqrt{d/h}}\bigg)(XV_i)
$$

其中 $Y_i\in\mathbb{R}^{\ell \times d/h}$，$\ell$ 为序列长度。

在我们的实现中，我们对注意力权重应用 dropout（尽管实际上可以在任何步骤使用）：

$$
Y_i = \text{dropout}\bigg(\text{softmax}\bigg(\frac{(XQ_i)(XK_i)^\top}{\sqrt{d/h}}\bigg)\bigg)(XV_i)
$$

最后，自注意力的输出是各头的拼接结果经过线性变换后的结果：

\begin{equation}
Y = [Y_1;\dots;Y_h]A
\end{equation}

其中 $A \in\mathbb{R}^{d\times d}$，$[Y_1;\dots;Y_h]\in\mathbb{R}^{\ell \times d}$。

在文件 `mml/transformer_layers.py` 中的 `MultiHeadAttention` 类中实现多头缩放点积注意力。下面的代码将检查你的实现，误差应小于 `e-3`。

In [ ]:
torch.manual_seed(231)

# Choose dimensions such that they are all unique for easier debugging:
# Specifically, the following values correspond to N=1, H=2, T=3, E//H=4, and E=8.
batch_size = 1
sequence_length = 3
embed_dim = 8
attn = MultiHeadAttention(embed_dim, num_heads=2)

# Self-attention.
data = torch.randn(batch_size, sequence_length, embed_dim)
self_attn_output = attn(query=data, key=data, value=data)

# Masked self-attention.
mask = torch.randn(sequence_length, sequence_length) < 0.5
masked_self_attn_output = attn(query=data, key=data, value=data, attn_mask=mask)

# Attention using two inputs.
other_data = torch.randn(batch_size, sequence_length, embed_dim)
attn_output = attn(query=data, key=other_data, value=other_data)

expected_self_attn_output = np.asarray([[
[-0.2494,  0.1396,  0.4323, -0.2411, -0.1547,  0.2329, -0.1936,
          -0.1444],
         [-0.1997,  0.1746,  0.7377, -0.3549, -0.2657,  0.2693, -0.2541,
          -0.2476],
         [-0.0625,  0.1503,  0.7572, -0.3974, -0.1681,  0.2168, -0.2478,
          -0.3038]]])

expected_masked_self_attn_output = np.asarray([[
[-0.1347,  0.1934,  0.8628, -0.4903, -0.2614,  0.2798, -0.2586,
          -0.3019],
         [-0.1013,  0.3111,  0.5783, -0.3248, -0.3842,  0.1482, -0.3628,
          -0.1496],
         [-0.2071,  0.1669,  0.7097, -0.3152, -0.3136,  0.2520, -0.2774,
          -0.2208]]])

expected_attn_output = np.asarray([[
[-0.1980,  0.4083,  0.1968, -0.3477,  0.0321,  0.4258, -0.8972,
          -0.2744],
         [-0.1603,  0.4155,  0.2295, -0.3485, -0.0341,  0.3929, -0.8248,
          -0.2767],
         [-0.0908,  0.4113,  0.3017, -0.3539, -0.1020,  0.3784, -0.7189,
          -0.2912]]])

print('self_attn_output error: ', rel_error(expected_self_attn_output, self_attn_output.detach().numpy()))
print('masked_self_attn_output error: ', rel_error(expected_masked_self_attn_output, masked_self_attn_output.detach().numpy()))
print('attn_output error: ', rel_error(expected_attn_output, attn_output.detach().numpy()))

# 位置编码 (Positional Encoding)

尽管 Transformers 可以轻松地关注输入的任意部分，但注意力机制本身并没有关于标记顺序的概念。然而，对于许多任务（尤其是自然语言处理），标记之间的相对顺序非常重要。为了弥补这一点，研究者在单词标记的嵌入上添加了位置编码。

我们定义一个矩阵 $P \in \mathbb{R}^{l\times d}$，其中 $P_{ij} = $

$$
\begin{cases}
\text{sin}\left(i \cdot 10000^{-\frac{j}{d}}\right) & \text{if j is even} \\
\text{cos}\left(i \cdot 10000^{-\frac{(j-1)}{d}}\right) & \text{otherwise} \\
\end{cases}
$$

而不是直接将输入 $X \in \mathbb{R}^{l\times d}$ 传递给我们的网络，我们传递的是 $X + P$。

在文件 `mml/transformer_layers.py` 中的 `PositionalEncoding` 类中实现这一层。完成后，运行以下代码以对你的实现进行简单测试。你应该看到误差在 `e-3` 量级或更小。

In [ ]:
torch.manual_seed(231)

batch_size = 1
sequence_length = 2
embed_dim = 6
data = torch.randn(batch_size, sequence_length, embed_dim)

pos_encoder = PositionalEncoding(embed_dim)
output = pos_encoder(data)

expected_pe_output = np.asarray([[[-1.2340,  1.1127,  1.6978, -0.0865, -0.0000,  1.2728],
                                  [ 0.9028, -0.4781,  0.5535,  0.8133,  1.2644,  1.7034]]])

print('pe_output error: ', rel_error(expected_pe_output, output.detach().numpy()))

# 基于 Transformer 的图像描述生成

现在你已经实现了之前的各个层，可以将它们组合起来构建一个基于 Transformer 的图像描述生成模型。打开文件 `mml/classifiers/transformer.py`，查看 `CaptioningTransformer` 类。

实现该类的 `forward` 函数。完成后，运行以下代码，使用一个小型测试用例检查你的前向传播实现；你应该看到误差在 `e-5` 量级或更小。

In [ ]:
torch.manual_seed(231)
np.random.seed(231)

N, D, W = 4, 20, 30
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}
V = len(word_to_idx)
T = 3

transformer = CaptioningTransformer(
    word_to_idx,
    input_dim=D,
    wordvec_dim=W,
    num_heads=2,
    num_layers=2,
    max_length=30
)

# Set all model parameters to fixed values
for p in transformer.parameters():
    p.data = torch.tensor(np.linspace(-1.4, 1.3, num=p.numel()).reshape(*p.shape))

features = torch.tensor(np.linspace(-1.5, 0.3, num=(N * D)).reshape(N, D))
captions = torch.tensor((np.arange(N * T) % V).reshape(N, T))

scores = transformer(features, captions)
expected_scores = np.asarray([[[-16.9532,   4.8261,  26.6054],
         [-17.1033,   4.6906,  26.4844],
         [-15.0708,   4.1108,  23.2924]],
        [[-17.1767,   4.5897,  26.3562],
         [-15.6017,   4.8693,  25.3403],
         [-15.1028,   4.6905,  24.4839]],
        [[-17.2172,   4.7701,  26.7574],
         [-16.6755,   4.8500,  26.3754],
         [-17.2172,   4.7701,  26.7574]],
        [[-16.3669,   4.1602,  24.6872],
         [-16.7897,   4.3467,  25.4831],
         [-17.0103,   4.7775,  26.5652]]])
print('scores error: ', rel_error(expected_scores, scores.detach().numpy()))

# 在小数据集上过拟合基于 Transformer 的描述生成模型

运行以下代码以在之前用于 RNN 的相同小数据集上对基于 Transformer 的描述生成模型进行过拟合。

In [ ]:
torch.manual_seed(231)
np.random.seed(231)

data = load_coco_data(max_train=50)

transformer = CaptioningTransformer(
          word_to_idx=data['word_to_idx'],
          input_dim=data['train_features'].shape[1],
          wordvec_dim=256,
          num_heads=2,
          num_layers=2,
          max_length=30
        )


transformer_solver = CaptioningSolverTransformer(transformer, data, idx_to_word=data['idx_to_word'],
           num_epochs=100,
           batch_size=25,
           learning_rate=0.001,
           verbose=True, print_every=10,
         )

transformer_solver.train()

# Plot the training losses.
plt.plot(transformer_solver.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training loss history')
plt.show()

打印最终的训练损失。你应该看到最终的损失小于 0.03。

In [ ]:
print('Final loss: ', transformer_solver.loss_history[-1])

# 测试时的 Transformer 采样

采样代码已经为你编写完成。你可以直接运行以下代码，与之前使用 RNN 的结果进行比较。如前所述，由于我们训练的数据量很少，训练集的结果应该比验证集的结果好得多。

In [ ]:
# If you get an error, the URL just no longer exists, so don't worry!
# You can re-sample as many times as you want.
for split in ['train', 'val']:
    minibatch = sample_coco_minibatch(data, split=split, batch_size=2)
    gt_captions, features, urls = minibatch
    gt_captions = decode_captions(gt_captions, data['idx_to_word'])

    sample_captions = transformer.sample(features, max_length=30)
    sample_captions = decode_captions(sample_captions, data['idx_to_word'])

    for gt_caption, sample_caption, url in zip(gt_captions, sample_captions, urls):
        img = image_from_url(url)
        # Skip missing URLs.
        if img is None: continue
        plt.imshow(img)            
        plt.title('%s\n%s\nGT:%s' % (split, sample_caption, gt_caption))
        plt.axis('off')
        plt.show()